In [1]:
import numpy as np
import matplotlib.pyplot as plt


/home/nbuser/anaconda2_501/lib/python2.7/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [49]:
def bionomial_model(u, d, R, S, n):
    '''S is the stock price at t = 0'''
    '''u and d are the multiplier with which the stock price "S" moves up and down respectively'''
    '''R is risk free interest rate'''
    '''n is the number of time periods until maturity'''
    
    #Calculate stock trr, ie. stock prices at every time interval
    stockTree = np.zeros((n+1, n+1))  
    stockTree[0,0] = S
    for i in range(1,n+1):
        stockTree[0,i] = stockTree[0, i-1]*u
        for j in range(1,n+1):
            stockTree[j,i] = stockTree[j-1, i-1]*d
    return stockTree
    

In [50]:
s = bionomial_model(1.06, 1/1.06, 1.02, 100, 3)
print(s)

[[100.         106.         112.36       119.1016    ]
 [  0.          94.33962264 100.         106.        ]
 [  0.           0.          88.999644    94.33962264]
 [  0.           0.           0.          83.9619283 ]]


In [51]:
def calculate_risk_neutral_prob(u, d, R):
    q1 = (R - d) / (u - d) 
    return q1
    

In [54]:
def european_call(u, d, R, S, n, K):
    '''u, d, R, S are described as above'''
    '''K is the strike price'''
    stockTree = bionomial_model(u, d, R, S, n)
    
    # compute the option tree
    optionTree = np.zeros((n+1,n+1))
    for j in range(n+1):
        optionTree[j, n] = max(0,  (stockTree[j, n]-K))
        
    q1 = calculate_risk_neutral_prob(u, d, R) 
    q2 = 1 - q1
    '''q1 and q2 are called risk neutral probabilities'''
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            optionTree[j, i] = (q1 * optionTree[j, i+1] + q2 * optionTree[j+1, i+1])/R
   
                
    return optionTree

In [55]:
eo = european_call(1.06, 1/1.06, 1.02, 100, 3, 95)
eo

array([[11.04359378, 14.76365993, 19.2227451 , 24.1016    ],
       [ 0.        ,  4.56317851,  7.08484041, 11.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [56]:
def european_call_price(u, d, R, S, n, K):
    optionTree = european_call(u, d, R, S, n, K)
    return optionTree[0][0]

In [57]:
print european_call_price(1.06, 1/1.06, 1.02, 100, 3, 95)
print european_call_price(1.06, 1/1.06, 1.04, 100, 3, 95)

### So the option price increases when we increase R.

11.043593784070934
15.637788819426035


In [58]:
def european_put(u, d, R, S, n, K):
    '''u, d, R, S are described as above'''
    '''K is the strike price'''
    stockTree = bionomial_model(u, d, R, S, n)
    
    # compute the option tree
    optionTree = np.zeros((n+1,n+1))
    for j in range(n+1):
        optionTree[j, n] = max(0,  (K - stockTree[j, n]))
        
    q1 = calculate_risk_neutral_prob(u, d, R)
    q2 = 1 - q1
    '''q1 and q2 are called risk neutral probabilities'''
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            optionTree[j, i] = (q1 * optionTree[j, i+1] + q2 * optionTree[j+1, i+1])/R
   
                
    return optionTree

In [59]:
def american_put (u, d, R, S, n, K):
    '''parameters remain same as defined above'''
    optionTree = european_put(u, d, R, S, n, K)
    stockTree = bionomial_model(u, d, R, S, n)
    q1 = (R - d) / (u - d) 
    q2 = 1 - q1
    
    print(stockTree)
    print("risk neutral probability is " + str(q1))
    print(optionTree)
    
    #checking if early excercise is a good option 
    flag = 0 
    list = []
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            optionTree[j, i] = max((q1 * optionTree[j, i+1] + q2 * optionTree[j+1, i+1])/R,   
                               (stockTree[j, i] - K))           
            #if the amount obtained by selling the option (Stock price - strike) is more than the oprion price, exercise early
            if (optionTree[j, i] - (stockTree[j, i] - K)) < 1e-10:
                flag += 1
                list.append(i)
    
    when = n
    if(flag):  when = list[-1]
    print("Can be exercised at time period " + str(when) + " with value " + str(optionTree[0][0]))
    return (optionTree[0,0], when)

In [60]:
american_put(1.07, 1/1.07 , 1.01, 100, 3, 100)

[[100.         107.         114.49       122.5043    ]
 [  0.          93.45794393 100.         107.        ]
 [  0.           0.          87.34387283  93.45794393]
 [  0.           0.           0.          81.62978769]]
risk neutral probability is 0.556935817805
[[ 3.63339827  1.25893933  0.          0.        ]
 [ 0.          6.7001215   2.8698522   0.        ]
 [ 0.          0.         11.66602816  6.54205607]
 [ 0.          0.          0.         18.37021231]]
Can be exercised at time period 2 with value 8.0393114659869


(8.0393114659869, 2)